In [ ]:
# ===== Eclipse SCL-BT – Cell 1: setup, imports, paths =====

!pip install -q nltk

import os
import json
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.model_selection import train_test_split

import nltk
nltk.download("punkt", quiet=True)  # not strictly needed, but harmless

from google.colab import drive
drive.mount("/content/drive")

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

# Root where your v02 folder lives
DATA_ROOT    = "/content/drive/MyDrive/msr2013-bug_dataset-master/data/v02"
ECLIPSE_DIR  = f"{DATA_ROOT}/eclipse"

ASSIGNED_FILE   = f"{ECLIPSE_DIR}/assigned_to.json"
SHORT_DESC_FILE = f"{ECLIPSE_DIR}/short_desc.json"
BUG_STATUS_FILE = f"{ECLIPSE_DIR}/bug_status.json"
RESOLUTION_FILE = f"{ECLIPSE_DIR}/resolution.json"

print("Eclipse folder:", ECLIPSE_DIR)
print("Files:")
print(" ", ASSIGNED_FILE)
print(" ", SHORT_DESC_FILE)
print(" ", BUG_STATUS_FILE)
print(" ", RESOLUTION_FILE)


Mounted at /content/drive
Using device: cuda
Eclipse folder: /content/drive/MyDrive/msr2013-bug_dataset-master/data/v02/eclipse
Files:
  /content/drive/MyDrive/msr2013-bug_dataset-master/data/v02/eclipse/assigned_to.json
  /content/drive/MyDrive/msr2013-bug_dataset-master/data/v02/eclipse/short_desc.json
  /content/drive/MyDrive/msr2013-bug_dataset-master/data/v02/eclipse/bug_status.json
  /content/drive/MyDrive/msr2013-bug_dataset-master/data/v02/eclipse/resolution.json


In [ ]:
# ===== Eclipse SCL-BT – Cell 2: build classifier-like DataFrame =====

import pandas as pd

def load_last_event_table(path, root_key, out_col):
    """
    MSR2013 JSON format:
      { root_key : { bug_id : [ {when, what, who}, ... ], ... } }

    For each bug_id we take the LAST event (max 'when') and keep 'what'.
    Returns DataFrame with columns: ['bug_id', out_col].
    """
    with open(path, "r", encoding="utf-8") as f:
        raw = json.load(f)

    inner = raw[root_key]
    rows = []

    for bug_id, events in inner.items():
        if not events:
            continue
        last = max(events, key=lambda e: e.get("when", 0))
        rows.append({
            "bug_id": int(bug_id),
            out_col: str(last.get("what", ""))
        })

    return pd.DataFrame(rows)

# 2.1 flatten JSONs
assigned_df   = load_last_event_table(ASSIGNED_FILE,   "assigned_to", "assigned_to")
short_df      = load_last_event_table(SHORT_DESC_FILE, "short_desc",  "short_desc")
bug_status_df = load_last_event_table(BUG_STATUS_FILE, "bug_status",  "bug_status")
resolution_df = load_last_event_table(RESOLUTION_FILE, "resolution",  "resolution")

print("assigned_df head:")
display(assigned_df.head())
print("short_df head:")
display(short_df.head())
print("bug_status_df head:")
display(bug_status_df.head())
print("resolution_df head:")
display(resolution_df.head())

# 2.2 merge on bug_id
df = (assigned_df
      .merge(short_df,      on="bug_id", how="inner")
      .merge(bug_status_df, on="bug_id", how="inner")
      .merge(resolution_df, on="bug_id", how="inner"))

print("Merged rows before filters:", len(df))

# 2.3 keep RESOLVED / VERIFIED / CLOSED AND FIXED
status_upper = df["bug_status"].str.upper()
res_upper    = df["resolution"].str.upper()

mask_status = status_upper.isin(["RESOLVED", "VERIFIED", "CLOSED"])
mask_res    = res_upper.str.contains("FIXED", na=False)

df = df[mask_status & mask_res].copy()
print("Rows after resolved+fixed filter:", len(df))

# 2.4 classifier-style columns
df["owner"]       = df["assigned_to"].astype(str)
df["issue_title"] = df["short_desc"].astype(str)
df["description"] = ""  # we only have short_desc here

# keep devs with at least MIN_BUGS reports (you were using >=2)
MIN_BUGS = 2
owner_counts = df["owner"].value_counts()
keep_owners  = owner_counts[owner_counts >= MIN_BUGS].index
df = df[df["owner"].isin(keep_owners)].reset_index(drop=True)

print("Rows after MIN_BUGS filter:", len(df))
print("Number of developers with ≥2 bugs:", df["owner"].nunique())

# 2.5 encode dev_id
df["dev_id"] = df["owner"].astype("category").cat.codes
n_devs = df["dev_id"].nunique()
print("Final number of developers:", n_devs)

clf_df = df[["owner", "issue_title", "description", "dev_id"]]
display(clf_df.head())


assigned_df head:


bug_id           assigned_to
0    1609  Darin_Wright@oti.com
1    1536  Darin_Wright@oti.com
2    1537  Darin_Wright@oti.com
3    1538  Darin_Wright@oti.com
4    1539  Darin_Wright@oti.com

short_df head:


bug_id                                         short_desc
0    3641         DCR: Can't compare from outliner (1GDHJKK)
1    3470  DCR: Can't compare code editions in type hiera...
2    3337  Open on selection failed with double message (...
3    3210  Search - method declarations within TypeHierar...
4    2968              TaskListMarker violates API (1GL2XHA)

bug_status_df head:


bug_id bug_status
0    4203   VERIFIED
1    4351   RESOLVED
2    3456     CLOSED
3    3475   VERIFIED
4    3469   RESOLVED

resolution_df head:


bug_id resolution
0    4203      FIXED
1    4351      FIXED
2    3456      FIXED
3    3475      FIXED
4    3469    WONTFIX

Merged rows before filters: 165547
Rows after resolved+fixed filter: 76268
Rows after MIN_BUGS filter: 76185
Number of developers with ≥2 bugs: 439
Final number of developers: 439


owner                                        issue_title  \
0     Darin_Wright@oti.com  classFilePattern breakpoint attribute optimiza...   
1      Jared_Burns@oti.com                       NPE in LocalEvaluationEngine   
2  Joe_Szurszewski@oti.com                   Extra Action delegates (1GIGUK0)   
3    Darin_Swanson@oti.com              HotSpot VM -server argument (1GLCPDH)   
4    Darin_Swanson@oti.com    Code Review: JDI debug model shutdown (1GLCWH3)   

  description  dev_id  
0                  23  
1                  40  
2                  46  
3                  20  
4                  20

In [ ]:
# ===== Eclipse SCL-BT – Cell 3: tokenize + build vocabulary =====

import re
from collections import Counter

# 3.1 combine title + description
clf_df["text"] = (
    clf_df["issue_title"].fillna("").astype(str)
    + " "
    + clf_df["description"].fillna("").astype(str)
)

# 3.2 regex tokenizer (same spirit as Chromium SCL-BT)
token_re = re.compile(r"[a-z]+")

def tokenize(text: str):
    text = str(text).lower()
    return token_re.findall(text)

print("Tokenising all Eclipse bug reports...")
clf_df["tokens"] = clf_df["text"].apply(tokenize)

# 3.3 build frequency vocabulary
MAX_VOCAB = 50000
MIN_FREQ  = 5

freq = Counter()
for toks in clf_df["tokens"]:
    freq.update(toks)

filtered_words = [w for w, c in freq.items() if c >= MIN_FREQ]
filtered_words.sort(key=lambda w: -freq[w])   # sort by frequency desc
filtered_words = filtered_words[:MAX_VOCAB]

word2id = {w: i for i, w in enumerate(filtered_words)}
id2word = {i: w for w, i in word2id.items()}
vocab_size = len(word2id)

print("Vocab size after cutoff:", vocab_size)

# 3.4 map tokens -> ids
def tokens_to_ids(tokens):
    return [word2id[w] for w in tokens if w in word2id]

clf_df["token_ids"] = clf_df["tokens"].apply(tokens_to_ids)

print(clf_df[["text", "tokens", "token_ids"]].head())


/tmp/ipython-input-3061412062.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clf_df["text"] = (


Tokenising all Eclipse bug reports...


/tmp/ipython-input-3061412062.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clf_df["tokens"] = clf_df["text"].apply(tokenize)


Vocab size after cutoff: 5366
                                                text  \
0  classFilePattern breakpoint attribute optimiza...   
1                      NPE in LocalEvaluationEngine    
2                  Extra Action delegates (1GIGUK0)    
3             HotSpot VM -server argument (1GLCPDH)    
4   Code Review: JDI debug model shutdown (1GLCWH3)    

                                              tokens  \
0  [classfilepattern, breakpoint, attribute, opti...   
1                   [npe, in, localevaluationengine]   
2                  [extra, action, delegates, giguk]   
3            [hotspot, vm, server, argument, glcpdh]   
4  [code, review, jdi, debug, model, shutdown, gl...   

                        token_ids  
0           [161, 447, 1952, 615]  
1                         [19, 0]  
2                [623, 135, 2647]  
3           [4825, 418, 909, 581]  
4  [33, 1044, 1370, 76, 166, 531]  


/tmp/ipython-input-3061412062.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clf_df["token_ids"] = clf_df["tokens"].apply(tokens_to_ids)


In [ ]:
# ===== Eclipse SCL-BT – Cell 4: padded sequences + splits =====

from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np

MAX_LEN = 300   # same as Chromium SCL-BT

sequences = clf_df["token_ids"].tolist()
X_all = pad_sequences(
    sequences,
    maxlen=MAX_LEN,
    padding="post",
    truncating="post"
)

y_all = clf_df["dev_id"].values.astype("int32")

print("X_all shape:", X_all.shape)
print("y_all shape:", y_all.shape)

bug_indices = np.arange(len(clf_df))

# 80% train, 20% test
train_idx, test_idx, y_train_tmp, y_test = train_test_split(
    bug_indices,
    y_all,
    test_size=0.2,
    random_state=42,
    stratify=y_all,
)

# From training part, 10% as validation
train_idx, val_idx, y_train, y_val = train_test_split(
    train_idx,
    y_train_tmp,
    test_size=0.1,     # 10% of 80% => 8% of total
    random_state=42,
    stratify=y_train_tmp,
)

print(f"Train bugs: {len(train_idx)}  Val bugs: {len(val_idx)}  Test bugs: {len(test_idx)}")

X_train = X_all[train_idx]
X_val   = X_all[val_idx]
X_test  = X_all[test_idx]

y_train = y_all[train_idx]
y_val   = y_all[val_idx]
y_test  = y_all[test_idx]

print("Train:", X_train.shape, " Val:", X_val.shape, " Test:", X_test.shape)


X_all shape: (76185, 300)
y_all shape: (76185,)
Train bugs: 54853  Val bugs: 6095  Test bugs: 15237
Train: (54853, 300)  Val: (6095, 300)  Test: (15237, 300)


In [ ]:
# ===== Eclipse SCL-BT – Cell 5: Torch datasets & dataloaders =====

from torch.utils.data import Dataset, DataLoader

BATCH_SIZE = 256

class BugDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.long)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = BugDataset(X_train, y_train)
val_dataset   = BugDataset(X_val,   y_val)
test_dataset  = BugDataset(X_test,  y_test)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,  drop_last=False)
val_loader   = DataLoader(val_dataset,   batch_size=BATCH_SIZE, shuffle=False, drop_last=False)
test_loader  = DataLoader(test_dataset,  batch_size=BATCH_SIZE, shuffle=False, drop_last=False)

print("Loaders ready.")


Loaders ready.


In [ ]:
# ===== Eclipse SCL-BT – Cell 6: model definition =====

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

EMBED_DIM  = 128
HIDDEN_DIM = 128
N_CLASSES  = n_devs         # from Cell 2
VOCAB_SIZE = vocab_size     # from Cell 3

class BugEncoder(nn.Module):
    """
    Embedding -> BiLSTM -> attention pooling -> bug representation z_b
    """
    def __init__(self, vocab_size, emb_dim, hidden_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        self.lstm = nn.LSTM(
            emb_dim,
            hidden_dim,
            batch_first=True,
            bidirectional=True,
        )
        self.attn = nn.Linear(2 * hidden_dim, 1)

    def forward(self, x):
        emb = self.embedding(x)            # [B, T, E]
        h, _ = self.lstm(emb)             # [B, T, 2H]

        scores = torch.tanh(self.attn(h)).squeeze(-1)   # [B, T]
        mask = (x != 0)                                  # [B, T]
        scores = scores.masked_fill(~mask, -1e9)

        alpha = torch.softmax(scores, dim=1)             # [B, T]
        z = torch.sum(h * alpha.unsqueeze(-1), dim=1)    # [B, 2H]
        return z


class SCLBTModel(nn.Module):
    """
    SCL-BT style:
      - Bug encoder -> z_b
      - Developer embedding table e_d
      - Classifier on top of z_b
    """
    def __init__(self, vocab_size, emb_dim, hidden_dim, n_classes):
        super().__init__()
        self.encoder = BugEncoder(vocab_size, emb_dim, hidden_dim)
        rep_dim = 2 * hidden_dim

        # Developer embedding table (not used directly in this code, but kept for consistency)
        self.dev_embed = nn.Embedding(n_classes, rep_dim)

        # Classifier
        self.classifier = nn.Linear(rep_dim, n_classes)

    def forward(self, x):
        z = self.encoder(x)               # [B, D]
        logits = self.classifier(z)       # [B, n_classes]
        return logits, z

model = SCLBTModel(VOCAB_SIZE, EMBED_DIM, HIDDEN_DIM, N_CLASSES).to(device)
print(model)


Using device: cuda
SCLBTModel(
  (encoder): BugEncoder(
    (embedding): Embedding(5366, 128, padding_idx=0)
    (lstm): LSTM(128, 128, batch_first=True, bidirectional=True)
    (attn): Linear(in_features=256, out_features=1, bias=True)
  )
  (dev_embed): Embedding(439, 256)
  (classifier): Linear(in_features=256, out_features=439, bias=True)
)


In [ ]:
# ===== Eclipse SCL-BT – Cell 7: SCL loss + training =====

def supervised_contrastive_loss(features, labels, temperature=0.1):
    """
    features: [B, D] float
    labels  : [B]   long
    """
    device_ = features.device
    labels = labels.contiguous().view(-1, 1)   # [B,1]
    batch_size = features.shape[0]

    mask = torch.eq(labels, labels.T).float().to(device_)
    logits_mask = torch.ones_like(mask) - torch.eye(batch_size, device=device_)
    mask = mask * logits_mask

    feats = F.normalize(features, dim=1)
    logits = torch.div(torch.matmul(feats, feats.T), temperature)

    logits_max, _ = torch.max(logits, dim=1, keepdim=True)
    logits = logits - logits_max.detach()

    exp_logits = torch.exp(logits) * logits_mask
    log_prob = logits - torch.log(exp_logits.sum(1, keepdim=True) + 1e-8)

    mask_sum = mask.sum(1)
    mask_sum = torch.where(mask_sum == 0, torch.ones_like(mask_sum), mask_sum)

    loss = -(mask * log_prob).sum(1) / mask_sum
    loss = loss.mean()
    return loss

LAMBDA_SCL = 0.2   # same as your Chromium run
EPOCHS     = 5
LR         = 1e-3

optimizer = torch.optim.Adam(model.parameters(), lr=LR)
best_val_loss = float("inf")
best_state = None

def run_epoch(loader, train=True):
    if train:
        model.train()
    else:
        model.eval()

    total_loss = 0.0
    total_ce   = 0.0
    total_scl  = 0.0
    total_cnt  = 0

    with torch.set_grad_enabled(train):
        for xb, yb in loader:
            xb = xb.to(device)
            yb = yb.to(device)

            logits, z = model(xb)
            loss_ce  = F.cross_entropy(logits, yb)
            loss_scl = supervised_contrastive_loss(z, yb)

            loss = loss_ce + LAMBDA_SCL * loss_scl

            if train:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            bs = xb.size(0)
            total_loss += loss.item() * bs
            total_ce   += loss_ce.item() * bs
            total_scl  += loss_scl.item() * bs
            total_cnt  += bs

    return (
        total_loss / total_cnt,
        total_ce   / total_cnt,
        total_scl  / total_cnt,
    )

for epoch in range(1, EPOCHS + 1):
    train_loss, train_ce, train_scl = run_epoch(train_loader, train=True)
    val_loss,   val_ce,   val_scl   = run_epoch(val_loader,   train=False)

    print(
        f"Epoch {epoch:02d} | "
        f"train_loss={train_loss:.4f} (CE={train_ce:.4f}, SCL={train_scl:.4f})  "
        f"val_loss={val_loss:.4f} (CE={val_ce:.4f}, SCL={val_scl:.4f})"
    )

    if val_loss < best_val_loss - 1e-4:
        best_val_loss = val_loss
        best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}

if best_state is not None:
    model.load_state_dict(best_state)
    model.to(device)

print("Training finished. Best val loss:", best_val_loss)


Epoch 01 | train_loss=5.2290 (CE=4.3834, SCL=4.2278)  val_loss=4.6820 (CE=3.8573, SCL=4.1234)
Epoch 02 | train_loss=4.3598 (CE=3.5410, SCL=4.0938)  val_loss=4.1983 (CE=3.3927, SCL=4.0278)
Epoch 03 | train_loss=3.9695 (CE=3.1667, SCL=4.0140)  val_loss=4.0204 (CE=3.2213, SCL=3.9951)
Epoch 04 | train_loss=3.7300 (CE=2.9406, SCL=3.9471)  val_loss=3.9340 (CE=3.1397, SCL=3.9716)
Epoch 05 | train_loss=3.5464 (CE=2.7643, SCL=3.9103)  val_loss=3.8937 (CE=3.1009, SCL=3.9640)
Training finished. Best val loss: 3.893706975519119


In [ ]:
# ===== Eclipse SCL-BT – Cell 8: evaluation Hit@K =====

model.eval()
all_logits = []
all_labels = []

with torch.no_grad():
    for xb, yb in test_loader:
        xb = xb.to(device)
        yb = yb.to(device)

        logits, _ = model(xb)
        all_logits.append(logits.cpu())
        all_labels.append(yb.cpu())

all_logits = torch.cat(all_logits, dim=0).numpy()
all_labels = torch.cat(all_labels, dim=0).numpy()

probs = torch.softmax(torch.from_numpy(all_logits), dim=1).numpy()

def hit_at_k(probs, y_true, k):
    topk = np.argsort(-probs, axis=1)[:, :k]
    hits = (topk == y_true.reshape(-1, 1)).any(axis=1)
    return float(hits.mean())

TOPKS = [1, 3, 5, 7, 9]
hrs = []

print("=== SCL-BT-style on Eclipse (bug encoder + supervised contrastive) ===")
for k in TOPKS:
    hr = hit_at_k(probs, all_labels, k)
    hrs.append(hr)
    print(f"Top-{k} Hit Ratio: {hr:.4f}")

avg_hr = float(np.mean(hrs))
print(f"Average Hit Ratio SCL-BT-style (Eclipse) : {avg_hr:.4f}")


=== SCL-BT-style on Eclipse (bug encoder + supervised contrastive) ===
Top-1 Hit Ratio: 0.3024
Top-3 Hit Ratio: 0.5127
Top-5 Hit Ratio: 0.6080
Top-7 Hit Ratio: 0.6646
Top-9 Hit Ratio: 0.7015
Average Hit Ratio SCL-BT-style (Eclipse) : 0.5578
